In [0]:
#regular expression to fa

state_count = -1

def add_transition(key,trans_state,transition):
    if transition.get(key):
        transition[key].append(trans_state)
    else:
        transition[key] = [trans_state]

def primitive(a):
    global state_count
    start = state_count +1
    final = start+1
    state_count+=2
    states = [start,final]
    #transition = {(start,a):[final]}
    transition = {}
    add_transition((start,a),final,transition)
    nfa = {"start":start,"final":final,
           "states":states,"transition":transition}
    return nfa

def union(nfa1,nfa2):
    global state_count
    start = state_count+1
    final = start+1
    state_count+=2
    states = nfa1['states'] + nfa2['states'] + [start,final]
    transition = {**nfa1["transition"],**nfa2["transition"]}
    add_transition((start,"E"),nfa1["start"],transition)
    add_transition((start,"E"),nfa2["start"],transition)
    add_transition((nfa1["final"],"E"),final,transition)
    add_transition((nfa2["final"],"E"),final,transition)
    
    #transition[(start,"E")].append(nfa1["start"])
    #transition[(start,"E")].append(nfa2["start"])
    #transition[(nfa1["final"],"E")].append(final)
    #transition[(nfa2["final"],"E")].append(final)
    
    nfa = {"start":start,"final":final,
           "states":states,"transition":transition}
    return nfa

def concatenation(nfa1,nfa2):
    global state_count
    start = nfa1["start"]
    final = nfa2["final"]
    states = nfa1["states"] + nfa2["states"]
    transition = {**nfa1["transition"],**nfa2["transition"]}
    #transition[(nfa1["final"],"E")].append(nfa2["start"])
    add_transition((nfa1["final"],"E"),nfa2["start"],transition)
    nfa = {"start":start,"final":final,
           "states":states,"transition":transition}
    return nfa
    
def kleeneclosure(nfa):
    global state_count
    start = state_count+1
    final = start+1
    state_count+=2
    states = nfa["states"] + [start,final]
    transition = nfa["transition"]
    add_transition((start,"E"),nfa["start"],transition)
    add_transition((nfa["final"],"E"),final,transition)
    add_transition((start,"E"),final,transition)
    add_transition((nfa["final"],"E"),nfa["start"],transition)

    #transition[(start,"E")].append(nfa["start"])    
    #transition[(nfa["final"],"E")].append(final)
    #transition[(start,"E")].append(final)
    #transition[(nfa["final"],"E")].append(nfa["start"])

    nfa = {"start":start,"final":final,
           "states":states,"transition":transition}
    return nfa

def infix_to_postfix(infix):
    operators = ["+",".","*"] #inorder of increasing precedence
    infix+=")"
    postfix = ""
    stack = ["("]
    cur = 0
    while(len(stack)):
        if infix[cur] == "(":
            stack.append("(")
        elif infix[cur] == ")":
            while(1):
                temp = stack.pop()
                if temp == "(":
                    break
                else:
                    postfix += temp                           
        elif not (infix[cur] in operators):
            postfix += infix[cur]
        else:
            while(1):
                temp = stack[-1]
                if temp =="(":
                    break
                else:
                    if (operators.index(temp)>operators.index(infix[cur])):
                        postfix += stack.pop()
                    else:
                        break
            stack.append(infix[cur])                        
        cur+=1  
    return postfix

def regexp_to_nfa(reg_exp):
    postfix = infix_to_postfix(reg_exp)
    operators = ["+",".","*"]
    stack = []
    for i in postfix:        
        if i not in operators:
            stack.append(primitive(i))
        else:
            if i=="*":
                a = stack.pop()
                stack.append(kleeneclosure(a))
            elif i=="+":
                b = stack.pop()
                a = stack.pop()
                stack.append(union(a,b))
            elif i==".":
                b = stack.pop()
                a = stack.pop()
                stack.append(concatenation(a,b))
    nfa = stack.pop()
    return nfa

reg_exp = "(a+b)+c*.d"
regexp_to_nfa(reg_exp)

{'final': 13,
 'start': 12,
 'states': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
 'transition': {(0, 'a'): [1],
  (1, 'E'): [5],
  (2, 'b'): [3],
  (3, 'E'): [5],
  (4, 'E'): [0, 2],
  (5, 'E'): [13],
  (6, 'c'): [7],
  (7, 'E'): [9, 6],
  (8, 'E'): [6, 9],
  (9, 'E'): [10],
  (10, 'd'): [11],
  (11, 'E'): [13],
  (12, 'E'): [4, 8]}}